In [1]:
import pandas as pd
import numpy as np

rawtuxdata = pd.read_pickle('config_bdd30-100.pkl')

basic_head = ["cid", "time", "date"] # "compile"
size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]

print("some configurations may have X86_32 (coz we have tested/tried some options and there are in the database) -- we removed them of course")
# we only keep X86_64 configurations
#rawtuxdata = rawtuxdata[rawtuxdata['X86_64'] == 'y'] ### TODO: I've impression it's not the most effective way (wrt memory) to filter 
if 'X86_64' in rawtuxdata.columns:
    print(rawtuxdata['X86_64'].describe())
    rawtuxdata.query("X86_64 == 'y'", inplace=True) # only keep them


some configurations may have X86_32 (coz we have tested/tried some options and there are in the database) -- we removed them of course
count     69459
unique        2
top           y
freq      69423
Name: X86_64, dtype: object


In [2]:
### TODO: ensure that the same configuration leads to the same compilation status (normally it's the case)
# remove entries with same configurations
print(str(len(rawtuxdata)) + " before the removal of some entries (those with same configurations)")
# tuxdata.drop_duplicates(subset=tuxdata.columns.difference(['vmlinux']), inplace=True)
rawtuxdata.drop_duplicates(subset=rawtuxdata.columns.difference(size_methods).difference(basic_head), inplace=True)
print(str(len(rawtuxdata)) + " after the removal of some entries (those with same configurations)")

69423 before the removal of some entries (those with same configurations)
69364 after the removal of some entries (those with same configurations)


In [3]:
cid_to_remove=74459 # compilation time is very low (2 sec)
cid_to_remove2=30698 # size is 0! 

rawtuxdata = rawtuxdata[rawtuxdata.cid != cid_to_remove]
rawtuxdata = rawtuxdata[rawtuxdata.cid != cid_to_remove2]

In [4]:
# assert (len(rawtuxdata) == 20001)
len(rawtuxdata)

69362

In [5]:
def nbyes(row):
    return sum(row == "y")                    
rawtuxdata['nbyes'] = rawtuxdata.apply(nbyes, axis=1)

In [6]:
rawtuxdata.sort_values(by="nbyes")[:10]

,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,nbyes
14299,74458,2018-07-05 16:07:21,28.7108,7317008,646608.0,2733176,501222,4722128.0,6808144,458568,...,n,y,n,n,n,n,n,n,n,240
16734,57051,2018-06-26 17:44:03,350.4990,28838328,6677456.0,8762488,6531115,10453968.0,12538448,6188913,...,n,y,y,n,n,n,n,n,n,668
4723,34765,2018-06-13 00:11:19,120.3810,64176376,3617744.0,5704408,3407623,7807952.0,9894056,3479187,...,n,y,n,n,n,n,n,n,n,684
20435,80598,2018-07-25 14:33:52,270.8170,14808224,2303440.0,4388304,2154587,6346192.0,8430504,2080144,...,n,y,y,n,n,n,n,n,n,708
14064,74219,2018-07-05 09:25:11,134.2110,13729408,3380176.0,5465552,3233544,7439312.0,9524136,3173721,...,m,y,n,n,n,n,n,m,n,710
13877,53941,2018-06-16 13:24:35,172.9770,25078328,2339792.0,4425168,2192433,6382544.0,8467368,2117897,...,n,y,n,y,n,n,n,n,n,713
1891,61891,2018-06-27 08:31:38,322.6870,65978424,2691024.0,4778824,2488385,6881232.0,8968480,2563987,...,n,y,n,n,n,n,n,n,n,715
20162,80325,2018-07-25 13:05:31,173.8940,12783848,2270160.0,4355536,2121304,6337488.0,8422312,2072046,...,m,y,n,n,n,n,n,m,n,718
1112,31115,2018-06-11 18:46:09,156.1740,73461752,3081168.0,5166552,2933658,7173072.0,9257896,2906856,...,m,y,n,n,n,n,n,m,n,730
14306,74465,2018-07-05 17:11:54,225.9910,48912544,6660048.0,8746104,6515085,10166224.0,12251728,5903503,...,n,y,y,n,n,m,n,n,n,730


In [7]:
# tinykernel
rawtuxdata.query("vmlinux == 7317008") #.iloc[0]

,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,nbyes
14299,74458,2018-07-05 16:07:21,28.7108,7317008,646608.0,2733176,501222,4722128.0,6808144,458568,...,n,y,n,n,n,n,n,n,n,240


In [8]:
rawtuxdata.query("vmlinux != -1").sort_values(by="vmlinux")[:20]

,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,nbyes
14299,74458,2018-07-05 16:07:21,28.7108,7317008,646608.0,2733176,501222,4722128.0,6808144,458568,...,n,y,n,n,n,n,n,n,n,240
6648,96648,2018-08-02 10:45:35,359.9280,10524528,2069456.0,4154488,1923864,6120400.0,8204880,1855777,...,n,y,y,n,n,m,n,n,n,857
22106,82269,2018-07-25 21:08:02,409.6480,10572520,2355152.0,4441552,2206376,6426576.0,8512424,2161455,...,n,y,n,n,n,n,n,n,n,816
764,30764,2018-06-11 16:52:17,253.7850,10707344,2126800.0,4212176,1978461,6181840.0,8266664,1916231,...,m,y,n,y,n,m,n,n,n,905
14992,55144,2018-06-21 02:21:50,152.9950,10786744,2716624.0,4801656,2571228,6759376.0,8843856,2494516,...,m,y,n,n,n,m,n,m,n,819
9653,49674,2018-06-15 02:05:06,329.4830,10790352,2678736.0,4764792,2534626,6709200.0,8794704,2445408,...,n,y,n,n,n,n,n,m,n,946
15561,75723,2018-07-07 12:03:34,248.0520,10856456,2420688.0,4506744,2277016,6500304.0,8585808,2239508,...,n,y,y,n,n,m,n,n,n,903
21509,81672,2018-07-25 18:11:49,204.6170,10857304,2428880.0,4514936,2284321,6533072.0,8618576,2269723,...,n,y,n,n,n,m,n,n,n,847
27873,88036,2018-07-30 10:26:37,253.2480,10865408,2323408.0,4408440,2176745,6403024.0,8487504,2136880,...,m,y,n,n,n,m,n,m,n,893
7088,97088,2018-08-02 13:08:35,440.1010,10866864,2528208.0,4613240,2383219,6611920.0,8696400,2346863,...,n,y,n,n,n,n,n,n,n,899


In [9]:
compilation_failures = len(rawtuxdata.query("vmlinux == -1").index)
compilation_successes = len(rawtuxdata.query("vmlinux != -1").index)
n_compilations = len(rawtuxdata.index)
assert(compilation_successes + compilation_failures == n_compilations)
compilation_failure_percentage = (compilation_failures / n_compilations) * 100
print("compilation failures:", compilation_failures, "out of", n_compilations, "(", compilation_failure_percentage, "%)")

compilation failures: 3507 out of 69362 ( 5.056082581240449 %)


In [10]:
# BUGS EXPLORATION
def bug_exploration():
    rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'")[['cid', 'vmlinux']]
    rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y'")[['cid', 'vmlinux']]
    rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y'")[['cid', 'vmlinux']]
    rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y'")[['cid', 'vmlinux']]
    rawtuxdata.query("GENERIC_ALLOCATOR == 'y' & DRM_VBOXVIDEO == 'y'")[['cid', 'vmlinux']]
    rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'm'")[['cid', 'vmlinux']]
    return rawtuxdata.query("DRM_VBOXVIDEO == 'y'")[['cid', 'vmlinux']]

#bug_exploration()


# y or n values
n_AIC7XXX_alone = len(rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'").index)
n_AIC79XX_alone = len(rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y'").index)
# masking effects!
n_AIC_both = len(rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y' & AIC7XXX_BUILD_FIRMWARE == 'y'").index)
n_AIC_or = len(rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y' | AIC79XX_BUILD_FIRMWARE == 'y'").index)
print("AIC7XXX_BUILD_FIRMWARE (without 79XX)", n_AIC7XXX_alone - n_AIC_both, "AIC79XX_BUILD_FIRMWARE (without X7XX)", n_AIC79XX_alone - n_AIC_both)
print("AIC7XXX_BUILD_FIRMWARE or AIC79XX_BUILD_FIRMWARE",  n_AIC_or)
print("Without the 2 major configuration bugs, caused by 2 individual options, it remains", len(rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index))
# masking effects!
# option WANXL_BUILD_FIRMWARE responsible of many bugs, except... 1
n_wanxl = len(rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y')").index)
print("WANXL_BUILD_FIRMWARE", n_wanxl)
# same as previously, but we only keep compilation failures... and surprise:
n_wanxl_failure = len(rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1)").index)
print("There is apparently one subtle case with WANXL_BUILD_FIRMWARE=y causing **NO** configuration bug (with VIDEO_RCAR_VIN)", n_wanxl_failure) # len(rawtuxdata.query("GENERIC_ALLOCATOR == 'y' & vmlinux == -1").index)

assert ((compilation_failures - (n_AIC_or + n_wanxl_failure)) == len(rawtuxdata.query("WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index))
print("still tracking", (compilation_failures - (n_AIC_or + n_wanxl_failure)), "configuration bugs")
print("let's now investigate GENERIC_ALLOCATOR")
len(rawtuxdata.query("GENERIC_ALLOCATOR == 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index), len(rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index)
#len(rawtuxdata.query("WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index)
#len(rawtuxdata.query("vmlinux == -1").index)

#n_AIC_or + n_wanxl

AIC7XXX_BUILD_FIRMWARE (without 79XX) 1073 AIC79XX_BUILD_FIRMWARE (without X7XX) 1109
AIC7XXX_BUILD_FIRMWARE or AIC79XX_BUILD_FIRMWARE 2687
Without the 2 major configuration bugs, caused by 2 individual options, it remains 820
WANXL_BUILD_FIRMWARE 419
There is apparently one subtle case with WANXL_BUILD_FIRMWARE=y causing **NO** configuration bug (with VIDEO_RCAR_VIN) 419
still tracking 401 configuration bugs
let's now investigate GENERIC_ALLOCATOR


(117, 284)

In [11]:
rawtuxdata.sort_values(by='cid')

,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,nbyes
0,30000,2018-06-09 09:49:20,428.014,50222120,7603152.0,9689768,7403313,11789264.0,13875328,7471985,...,n,y,n,n,n,y,n,n,n,1435
1,30001,2018-06-09 09:49:41,241.167,16660024,4109776.0,6194296,3964503,8160720.0,10244688,3898077,...,n,y,y,n,n,m,m,m,n,1382
2,30002,2018-06-09 09:49:58,198.649,43080856,11191760.0,13276280,11046541,15177168.0,17261136,10912607,...,n,y,n,y,n,n,n,n,n,1626
3,30003,2018-06-09 09:50:05,214.034,27261672,8053712.0,10139088,7907068,11850704.0,13935528,7584629,...,y,y,y,y,n,y,n,y,n,2140
4,30004,2018-06-09 09:50:10,348.474,58769440,16135120.0,18220496,15986281,20018128.0,22102952,15752717,...,n,y,n,y,n,y,n,y,y,2651
5,30005,2018-06-09 09:50:17,658.585,24234952,7660496.0,9745872,7513697,11523024.0,13607848,7256145,...,n,y,y,n,m,m,n,m,y,1591
6,30006,2018-06-09 09:50:18,248.957,28801640,8741840.0,10827216,8592309,12489680.0,14574504,8224327,...,y,y,y,n,n,y,n,y,n,2115
7,30007,2018-06-09 09:50:40,545.106,109670360,30222096.0,32309128,30054546,32032528.0,34119016,27744837,...,n,y,n,n,n,y,n,y,y,3301
8,30008,2018-06-09 09:50:41,166.481,153521616,5800912.0,7886296,5651963,9831376.0,11916200,5566064,...,n,y,n,n,n,n,n,n,n,1510
9,30009,2018-06-09 09:50:45,244.012,30553608,8278992.0,10364024,8131211,12129232.0,14213712,7865559,...,y,y,n,n,n,y,n,n,y,2507


In [12]:
# Juliana 
rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'").sample(n=10)['cid'].values
len(rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'").index)

# Djamel 
rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y'").sample(n=10)['cid'].values
len(rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y'").index)

# Arnaud 
rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y')").sample(n=10)['cid'].values
rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y'").sample(n=10)['cid'].values

# Mathieu 
# GENERIC_ALLOCATOR == 'y' + DRM_VBOXVIDEO
# interestingly: 'm' (module) does not lead to a bug! < 1.5
# ['vmlinux', 'cid']
rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y'").sample(n=10)['cid'].values

array([43471, 72121, 85136, 49050, 61556, 67081, 58727, 50359, 44584,
       98902])

In [13]:
# masking effect
# Mathieu
print(rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & AIC79XX_BUILD_FIRMWARE == 'y'"))
print(rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & AIC7XXX_BUILD_FIRMWARE == 'y'"))
rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE == 'y'")

         cid                 date      time  vmlinux  GZIP-bzImage  \
757    40757  2018-06-14 02:00:56   661.035       -1          -1.0   
18936  59253  2018-06-27 00:25:34   533.816       -1          -1.0   
1758   61758  2018-06-27 08:06:58   545.480       -1          -1.0   
7576   67576  2018-06-28 04:21:58   275.427       -1          -1.0   
24005  84168  2018-07-26 16:14:35  2448.580       -1          -1.0   

       GZIP-vmlinux  GZIP  BZIP2-bzImage  BZIP2-vmlinux  BZIP2  ...    \
757              -1    -1           -1.0             -1     -1  ...     
18936            -1    -1           -1.0             -1     -1  ...     
1758             -1    -1           -1.0             -1     -1  ...     
7576             -1    -1           -1.0             -1     -1  ...     
24005            -1    -1           -1.0             -1     -1  ...     

       APDS9960  ARCH_SUPPORTS_INT128  SLABINFO  MICROCODE_AMD  \
757           m                     y         n              n   
18936   

,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,nbyes
6303,36364,2018-06-13 17:41:09,1300.540,-1,-1.0,-1,-1,-1.0,-1,-1,...,n,y,n,n,n,y,n,y,n,3081
13213,53234,2018-06-15 09:45:50,425.066,-1,-1.0,-1,-1,-1.0,-1,-1,...,n,y,n,y,n,y,n,y,y,2780


In [14]:
### basic stats about options and unique values 
## could be extended/improved 

tri_state_values = ['y', 'n', 'm']

ftuniques = []
freq_ymn_features = []
non_tristate_options = []

for col in rawtuxdata:
    ft = rawtuxdata[col]    
    # eg always "y"
    if len(ft.unique()) == 1:
        ftuniques.append(col)
    # only tri-state values (y, n, m) (possible TODO: handle numerical/string options)    
    elif all(x in tri_state_values for x in ft.unique()):     #len(ft.unique()) == 3: 
        freq = ft.value_counts(normalize=True)
        freqy = 0
        freqn = 0
        freqm = 0
        if ('y' in freq.index):
            freqy = freq['y']
        if ('n' in freq.index):
            freqn = freq['n']
        if ('m' in freq.index):
            freqm = freq['m']
        freq_ymn_features.append((col, freqy, freqm, freqn))
    else:
        if not (col in size_methods): 
            non_tristate_options.append(col)
        

### TODO: we want to keep all quantitative values!
# non_tristate_options.remove('LZO') # ('vmlinux')

# we want to keep measurements (that are not tristate ;)) 
# non_tristate_options = list(set(non_tristate_options) - set(size_methods))

#### print options with unique values
# options with only one value eg always "y"
#i = 0
#for ft in ftuniques:
#    print(ft + " (" + str(i) + ")")
#    i = i + 1

print("Original size (#configs/#options) of the dataset " + str(rawtuxdata.shape))
print ("Number of options with only one value (eg always y): " + str(pd.DataFrame(ftuniques).shape))

# maybe we can drop options with only one unique value (no interest for machine learning)
# TODO: maybe we can rely on more traditional feature reduction techniques
# TODO: need to think about *when* to apply the removal 

#rawtuxdata.drop(columns=ftuniques,inplace=True) 

## non_tristate_options include basic stuff like date, time, cid but also string/numerical options
print ("Non tri-state value options (eg string or integer or hybrid values): " 
       + str(pd.DataFrame(non_tristate_options).shape) + " ") 
#      + str(pd.DataFrame(non_tristate_options)))


print ("Predictor variables: " + str(rawtuxdata.drop(columns=non_tristate_options).columns.size))
# frequency of y, m, and n values 
#plt.figure()
#pd.DataFrame(freq_ymn_features, columns=["feature", "freqy", "freqm", "freqn"]).plot(kind='hist', alpha=0.8) #plot()
#plt.show()


rawtuxdata.info(memory_usage='deep')

Original size (#configs/#options) of the dataset (69362, 12799)
Number of options with only one value (eg always y): (3395, 1)
Non tri-state value options (eg string or integer or hybrid values): (147, 1) 
Predictor variables: 12652
<class 'pandas.core.frame.DataFrame'>
Int64Index: 69362 entries, 0 to 9999
Columns: 12799 entries, cid to nbyes
dtypes: float64(30), int64(235), object(12534)
memory usage: 53.5 GB


In [15]:
from sklearn.preprocessing import *
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn import tree
import graphviz 


LEARN_COMPILATION_SUCCESS = True # costly in time and space 
compilation_status_column_name = 'compile_success'

def encode_data_compilation(rawtuxdata):
    lae = LabelEncoder()
    # we save quantitative values we want (here vmlinux, TODO: generalize)
    # the key idea is that the labelling encoder should not be applied to this kind of values (only to predictor variables!)
    # vml = rawtuxdata['LZO'] # rawtuxdata['vmlinux'] 
    o_sizes = rawtuxdata[size_methods]

    # we may remove non tri state options, but TODO there are perhaps some interesting options (numerical or string) here
    #tuxdata = rawtuxdata.drop(columns=non_tristate_options).drop(columns=['vmlinux']).apply(le.fit_transform)
    tuxdata_for_compilation = rawtuxdata.drop(columns=non_tristate_options).drop(columns=size_methods).apply(lae.fit_transform)

    #tuxdata['vmlinux'] = vml 
    tuxdata_for_compilation[size_methods] = o_sizes
    # we can ue vmlinux since it has been restored thanks to previous line
    tuxdata_for_compilation[compilation_status_column_name] = tuxdata_for_compilation['vmlinux'] != -1
    return tuxdata_for_compilation

def learn_compilation_success(tuxdata_for_compilation):
    TESTING_SIZE=0.1 
    X_train, X_test, y_train, y_test = train_test_split(tuxdata_for_compilation.drop(columns=size_methods).drop(columns=compilation_status_column_name), tuxdata_for_compilation[compilation_status_column_name], test_size=TESTING_SIZE, random_state=0)  
    clf = tree.DecisionTreeClassifier() #GradientBoostingClassifier(n_estimators=100) #RandomForestRegressor(n_estimators=100) #   #GradientBoostingRegressor(n_estimators=100)  
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    importances = clf.feature_importances_
    indices = np.argsort(importances)[::-1]    

    TOP_FT_IMPORTANCE=20
    print("Feature ranking: " + "top (" + str(TOP_FT_IMPORTANCE) + ")")
    for f in range(TOP_FT_IMPORTANCE): # len(indices)
        print("%d. feature %s %d (%f)" % (f + 1, tuxdata_for_compilation.columns[indices[f]], indices[f], importances[indices[f]]))
   
    
    dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=tuxdata_for_compilation.drop(columns=size_methods).drop(columns=compilation_status_column_name).columns,  
                         filled=True, rounded=True,
                         special_characters=True)  
    graph = graphviz.Source(dot_data)  
    graph.render("TUXML_compilation_failures")
    
    acc = accuracy_score (y_test, y_pred)
    prec = precision_score (y_test, y_pred)
    reca = recall_score (y_test, y_pred)
    f1 = f1_score (y_test, y_pred)
    print("Accuracy score: %.2f" % (acc))
    print("Precision score: %.2f" % (prec))
    print("Recall score: %.2f" % (reca))
    print("F1 score: %.2f" % (f1))

if (LEARN_COMPILATION_SUCCESS):
    tuxdata_for_compilation = encode_data_compilation(rawtuxdata)
    # tuxdata_for_compilation [compilation_status_column_name].describe() # TODO?
    learn_compilation_success(tuxdata_for_compilation)
    del tuxdata_for_compilation

Feature ranking: top (20)
1. feature AIC79XX_BUILD_FIRMWARE 2547 (0.446314)
2. feature AIC7XXX_BUILD_FIRMWARE 1799 (0.312737)
3. feature WANXL_BUILD_FIRMWARE 10533 (0.120058)
4. feature DRM_VBOXVIDEO 8022 (0.077243)
5. feature IPV6 1792 (0.007983)
6. feature GENERIC_ALLOCATOR 11935 (0.005936)
7. feature FORTIFY_SOURCE 5353 (0.005300)
8. feature UBSAN_ALIGNMENT 2738 (0.003449)
9. feature INFINIBAND_ADDR_TRANS 6324 (0.003056)
10. feature VIDEO_V4L2 11661 (0.001530)
11. feature UBSAN_NULL 2745 (0.001424)
12. feature DRM_I915_WERROR 5003 (0.000636)
13. feature CUSE 7767 (0.000621)
14. feature MAX1363 7046 (0.000605)
15. feature SCSI_SRP_ATTRS 346 (0.000589)
16. feature VIDEO_MUX 12571 (0.000430)
17. feature DRM_MGA 12293 (0.000418)
18. feature PARIDE_EPIA 3022 (0.000331)
19. feature VIDEO_STI_BDISP 2162 (0.000330)
20. feature CHARGER_AXP20X 11500 (0.000330)
Accuracy score: 1.00
Precision score: 1.00
Recall score: 1.00
F1 score: 1.00
